In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from transformers import AutoTokenizer

In [3]:
BASE_MODEL_NAME='monologg/koelectra-small-v3-discriminator'

In [4]:
base_tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)

In [5]:
from src.tokenizer.modeling_tokenizer import BERTCharTokenizer

In [6]:
tokenizer = BERTCharTokenizer(
    BASE_MODEL_NAME,
    '[SEP]',
    'kor'
)

In [7]:
tokenizer.decode_char(*tokenizer.encode_char('안녕하세요. 그땐 왜 그랬어요?'))

'안녕하세요. 그땐 왜 그랬어요?'

In [8]:
tokenizer.encode_char("안녕하세요. 그땐 왜 그랬어요?")

([3081,
  2293,
  3755,
  2926,
  3183,
  18,
  3,
  2126,
  2469,
  3,
  3178,
  3,
  2126,
  2541,
  3114,
  3183,
  35],
 [1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0])

In [9]:
base_tokenizer.decode([3])

'[SEP]'

In [33]:
char_start, char_end = 0xAC00, 0xD7A3  # 가-힣
kor_chars = list(set([chr(code) for code in range(char_start, char_end + 1)]))
char_ids = []
for kor_char in kor_chars:
    ids = tokenizer.encode(kor_char, add_special_tokens=False)
    print(ids)


[1]
[2022]
[1]
[1]
[1]
[3313]
[1]
[1]
[1]
[1]
[1]
[1]
[3865]
[1]
[1]
[1]
[1]
[1]
[2990]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[2205]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[2299]
[1]
[2835]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[3697]
[1]
[1]
[1]
[1]
[1]
[2422]
[1]
[1]
[2128]
[1]
[1]
[1]
[3491]
[1]
[1]
[1]
[1]
[2531]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[2742]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[3198]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[2900]
[1]
[1]
[2170]
[1]
[1]
[1]
[1]
[1]
[1]
[3010]
[2206]
[1]
[3728]
[1]
[1]
[1]
[1]
[1]
[1]
[2140]
[1]
[1]
[2695]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[2689]
[1]
[1]
[1]
[1]
[1]
[1]
[3742]
[1]
[2301]
[1]
[1]
[1]
[1]
[2544]
[1]
[1]
[1]
[1]
[2727]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[3686]
[1]
[1]
[1]
[1]
[1]
[2706]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[2174]
[1]
[1]
[3488]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[3690]
[3518]
[2042]
[1]
[2716]
[1]
[1]
[1

In [26]:
chos = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
joongs = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
jongs = [tokenizer.pad_token, 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
jamos = list(set(chos) | set(joongs) | set(jongs))
jamo_ids = tokenizer(jamos, add_special_tokens=False)['input_ids']
jamo_to_id = {jamo: jamo_id[-1] for jamo, jamo_id in zip(jamos, jamo_ids)}
cho_ids = [jamo_to_id[cho] for cho in chos]
joong_ids = [jamo_to_id[joong] for joong in joongs]
jong_ids = [jamo_to_id[jong] for jong in jongs]
id_to_jamo = {jamo_id: jamo for jamo, jamo_id in jamo_to_id.items()}

In [27]:
id_to_jamo

{3: 'ㄺ',
 184: 'ㄷ',
 195: 'ㅜ',
 190: 'ㅈ',
 196: 'ㅠ',
 197: 'ㅡ',
 198: 'ㅣ',
 183: 'ㄴ',
 194: 'ㅏ',
 1: '[PAD]',
 189: 'ㅇ',
 182: 'ㄱ',
 188: 'ㅅ',
 191: 'ㅋ',
 186: 'ㅁ',
 192: 'ㅌ',
 185: 'ㄹ',
 187: 'ㅂ',
 193: 'ㅎ'}

In [24]:
max_len = -1
for hanzi_char in hanzi_chars:
    ids = tokenizer.encode(hanzi_char, add_special_tokens=False)
    if len(ids)>max_len:
        max_len=len(ids)

In [26]:
len(hanzi_chars)

88464

In [13]:
loss ,logits, pred_ids, sentence_denoised = krop.forward(batch['sentence'], batch['sentence_noisy'], task='char', pred=True)

In [12]:
batch

{'sentence': ['퓨갸 녀뮤 좋야용! 짤 뇰코 캅님따!!',
  '쳬갑 친냔 1월인갼? 2월춤웨 캅봤는톄, 67쯩엣써 슉쏢룰 잡앗셔오. 껴씰톡 클렇교 팡됴 엶쩡 넓꾜, 씹섧또 깩꿋햐교 부까 진쨔 끝넥줍뉘댜. 굵립꼬 억퀵갸 윌폰 슥쉼맞섦뭬셧 낢싣갸 좋우먼 뵤인닳꼬 하눼옴. 윕엑 100쭝엣 쓰탸뻑습또 졍맑 좋교, 헤욀옇행 깐눈 겄 까뜬 눅뀌뮈 뜨렁써오. 큰떼 쌀착 았쒸운 겡 옐씨튐 뮈떼 샹갸뚤뤼 꺼읠 공쉬릴예옷ㅠㅠ 굻련뎨 웠텨퍅끄됴 엶청 꾸케 잊덞람꾜욘. 삯윤났동 잇코, 엎웨 햇윤때 헷쑵욕쟝또 잇썽섟 청맒 좋야써옷. 굻릴교 웰싶틴옛셔 푿싼 룟뎨얼튼 뇰익귁쿡카 뵤잎뎌라곰요.'],
 'sentence_noisy': ['뷰가 너무 좋아요! 잘 놀고 갑니다!!',
  '제가 지난 1월인가? 2월쯤에 가봤는데, 67층에서 숙소를 잡았어요. 거실도 그렇고 방도 엄청 넓고, 시설도 깨끗하고 뷰가 진짜 끝내줍니다. 그리고 여기가 일본 스시마섬에서 날씨가 좋으면 보인다고 하네요. 위에 100층에 스타벅스도 정말 좋고, 해외여행 가는 것 같은 느낌이 들었어요. 근데 살짝 아쉬운 게 엘시티 밑에 상가들이 거의 공실이에요ㅠㅠ 그런데 워터파크도 엄청 크게 있더라고요. 사우나도 있고, 옆에 해운대 해수욕장도 있어서 정말 좋았어요. 그리고 엘시티에서 부산 롯데월드 놀이기구가 보이더라고요.']}

In [14]:
sentence_denoised

['됸뛧!쵧칧!튋녈쪉!!햋!맀녤!볍튥칟!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!',
 '뜃껻!뻘쨄!!쾗츻킥!!!뭻긿렷!눻륄뗐겴!!!!믾턬궐!웄셓쫘!턻츙퉑싶!!우촢럿!껬꿏엜!규룗!뵨뜅!뵙웄!!뜗섗혖!뵗햬맹웑!쨬탻!뙑섓!턓늣큄쾄룓!!칙퉗톺!퓽밨룗!윱섓!쨣옼퍙높뉍셤!벘풗퉑!뗑뜓켇!퍣펮꽓껮!줫팙뜙!!좔쉡!!!!뵧헭!먭퉧륤툓꽣!푯짊!닓턙!!쯩븡덣뵣!껙좇!뼣!킷턤!헭뙙툣!큒섓궤케!!떙슴!낔젋!뗤떭뗙!빣!앍뭃옫!턤껣!덙뜄렝뵣!끣먤!땣즓꾒섙녜!!!떙끭뺤!쌜뼓엾왜뙓!씐흴!즧쉥!띭뭙듷럭펏!!턮뚀감꽣!휈홂!!뼙꾤!섪뵓퓁!셓젒먈팣썻!뜻왙뗤!튀팓!엷퍒쉥뙧!!툘꾤샙!맞먮떓퉑뭤!띷끮!뭗큫낑옄!짫덒툈쨪숵!눪뮘팣뗀칣뚓!']